In [88]:
import json
import re

def extract_signals(activity_text):
    usb_outside_hours = False
    odd_hours_logins = False
    external_email = False
    large_attachments = False
    sensitive_file_access = False
    job_hunting_website_access = False
    leak_website_access = False
    job_hunting_email = False
    discontent_email = False

    job_sites = [
        "indeed.com", "glassdoor.com", "monster.com",
        "raytheon.com", "lockheedmartin.com", "jobhuntersbible.com",
        "careerbuilder.com", "simplyhired.com", "job-hunt.org"
    ]
    leak_sites = ["wikileaks", "pastebin", "anonfiles", "leak"]
    sensitive_keywords = ["salary", "resign", "exit", "interview", "confidential", "resume", "notice"]
    discontent = ["work not appreciated", "toxic environment", "not valued", "quitting soon", "burned out"]
    job_hunting = ["resume", "degree", "responsibilities", "salary", "benefits", "recruiter"]

    org_domain = "dtaa.com"

    for line in activity_text.splitlines():
        line_lower = line.strip().lower()

        # Extract hour from lines like "... at HH:MM ..."
        time_match = re.search(r'at (\d{1,2}):(\d{2})', line)
        hour = int(time_match.group(1)) if time_match else None
        is_outside = "outside business hours" in line_lower

        # USB activity
        if "usb device" in line_lower and (is_outside or (hour is not None and (hour < 6 or hour >= 18))):
            usb_outside_hours = True

        # Login outside hours
        if "logged in" in line_lower and (is_outside or (hour is not None and (hour < 6 or hour >= 18))):
            odd_hours_logins = True

        # External email domain
        if "Sent an email to" in line:
            email_match = re.search(r'to ([\w\.-]+@[\w\.-]+)', line, re.IGNORECASE)
            if email_match:
                recipient_email = email_match.group(1).lower()
                if not recipient_email.endswith(f"@{org_domain}"):
                    external_email = True

            # Large attachments
            if "large attachment" in line_lower:
                large_attachments = True

        # Email content analysis
        if "email content:" in line_lower:
            if any(keyword in line_lower for keyword in job_hunting):
                job_hunting_email = True
            if any(phrase in line_lower for phrase in discontent):
                discontent_email = True
            if any(keyword in line_lower for keyword in sensitive_keywords):
                sensitive_file_access = True  # treating sensitive keywords in email as "file access" too

        # File access line (for non-email sensitive access)
        if "accessed file" in line_lower:
            if any(keyword in line_lower for keyword in sensitive_keywords):
                sensitive_file_access = True

        # Website visits
        if "visited website" in line_lower:
            if any(site in line_lower for site in job_sites):
                job_hunting_website_access = True
            if any(site in line_lower for site in leak_sites):
                leak_website_access = True

    return {
        "usb_outside_hours": usb_outside_hours,
        "odd_hours_logins": odd_hours_logins,
        "external_email_usage": external_email,
        "large_attachments": large_attachments,
        "sensitive_file_access": sensitive_file_access,
        "job_hunting_website_access": job_hunting_website_access,
        "leak_website_access": leak_website_access,
        "job_hunting_email": job_hunting_email,
        "discontent_email": discontent_email
    }

def generate_explanation(user_id, risk_level, signals):
    reasons = []

    if signals["usb_outside_hours"]:
        reasons.append("connected a USB device outside business hours")
    if signals["odd_hours_logins"]:
        reasons.append("logged in during unusual hours")
    if signals["external_email_usage"]:
        reasons.append("sent email to a personal address")
    if signals["large_attachments"]:
        reasons.append("included large attachments in emails")
    if signals["sensitive_file_access"]:
        reasons.append("accessed sensitive or confidential files")
    if signals.get("job_hunting_website_access"):
        reasons.append("visited job-hunting websites")
    if signals.get("leak_website_access"):
        reasons.append("visited data leak or whistleblower websites")
    if signals["job_hunting_email"]:
        reasons.append("included job-hunting language in emails")
    if signals["discontent_email"]:
        reasons.append("expressed dissatisfaction or discontent in emails")

    if not reasons:
        return f"Lets think step by step. User {user_id} is flagged as {risk_level}. No suspicious behavior was detected."

    if len(reasons) == 1:
        reason_str = reasons[0]
    else:
        reason_str = ', '.join(reasons[:-1]) + ', and ' + reasons[-1]

    base = f"Lets think step by step. User {user_id} is flagged as {risk_level} because they {reason_str}."

    if risk_level == "Low":
        return base + " These actions may be explainable and do not indicate clear malicious intent."

    return base + " These actions align with patterns of insider threat behavior."

# === Load training Data ===
with open("training_data.json", "r", encoding="utf-8") as f:
    lines = [json.loads(line) for line in f if line.strip()]

knowledge_base = []

for item in lines:
    user_message = next((m["content"] for m in item["messages"] if m["role"] == "user"), "")
    assistant_message = next((m["content"] for m in item["messages"] if m["role"] == "assistant"), "")

    # Extract user ID
    user_id_match = re.search(r'User:\s*(\w+)', user_message)
    user_id = user_id_match.group(1) if user_id_match else "unknown"

    # Extract activity section
    activities_match = re.search(r'Activities:\s*(.*)', user_message, re.DOTALL)
    activity_section = activities_match.group(1).strip() if activities_match else ""

    # Extract risk level
    risk_match = re.search(r'Risk Level\s*:\s*(High|Low)', assistant_message, re.IGNORECASE)
    risk_level = risk_match.group(1).capitalize() if risk_match else "Unknown"

    # Extract signals and explanation
    signals = extract_signals(activity_section)
    explanation = generate_explanation(user_id, risk_level, signals)

    knowledge_base.append({
        "user_id": user_id,
        "summary": activity_section,
        "signals": signals,
        "risk_level": risk_level,
        "explanation": explanation
    })

# === Save to knowledge_base.json ===
with open("knowledge_base.json", "w", encoding="utf-8") as f:
    json.dump(knowledge_base, f, indent=2)

print(f"Extracted {len(knowledge_base)} entries and saved to 'knowledge_base.json'")

Extracted 80 entries and saved to 'knowledge_base.json'


In [94]:
import json
import os
import re
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

# === Step 1: Signal Extractor ===
def extract_signals(activity_text):
    usb_outside_hours = False
    odd_hours_logins = False
    external_email = False
    large_attachments = False
    sensitive_file_access = False
    job_hunting_website_access = False
    leak_website_access = False
    job_hunting_email = False
    discontent_email = False

    job_sites = [
        "indeed.com", "glassdoor.com", "monster.com",
        "raytheon.com", "lockheedmartin.com", "jobhuntersbible.com",
        "careerbuilder.com", "simplyhired.com", "job-hunt.org"
    ]
    leak_sites = ["wikileaks", "pastebin", "anonfiles", "leak"]
    sensitive_keywords = ["salary", "resign", "exit", "interview", "confidential", "resume", "notice"]
    discontent = ["work not appreciated", "toxic environment", "not valued", "quitting soon", "burned out"]
    job_hunting = ["resume", "degree", "responsibilities", "salary", "benefits", "recruiter"]

    org_domain = "dtaa.com"

    for line in activity_text.splitlines():
        line_lower = line.strip().lower()

        time_match = re.search(r'at (\d{1,2}):(\d{2})', line)
        hour = int(time_match.group(1)) if time_match else None
        is_outside = "outside business hours" in line_lower

        if "usb device" in line_lower and (is_outside or (hour is not None and (hour < 6 or hour >= 18))):
            usb_outside_hours = True

        if "logged in" in line_lower and (is_outside or (hour is not None and (hour < 6 or hour >= 18))):
            odd_hours_logins = True

        if "sent an email to" in line.lower():
            email_match = re.search(r'to ([\w\.-]+@[\w\.-]+)', line, re.IGNORECASE)
            if email_match:
                recipient_email = email_match.group(1).lower()
                if not recipient_email.endswith(f"@{org_domain}"):
                    external_email = True
            if "large attachment" in line_lower:
                large_attachments = True

        if "email content:" in line_lower:
            if any(keyword in line_lower for keyword in job_hunting):
                job_hunting_email = True
            if any(phrase in line_lower for phrase in discontent):
                discontent_email = True
            if any(keyword in line_lower for keyword in sensitive_keywords):
                sensitive_file_access = True

        if "accessed file" in line_lower:
            if any(keyword in line_lower for keyword in sensitive_keywords):
                sensitive_file_access = True

        if "visited website" in line_lower:
            if any(site in line_lower for site in job_sites):
                job_hunting_website_access = True
            if any(site in line_lower for site in leak_sites):
                leak_website_access = True

    return {
        "usb_outside_hours": usb_outside_hours,
        "odd_hours_logins": odd_hours_logins,
        "external_email_usage": external_email,
        "large_attachments": large_attachments,
        "sensitive_file_access": sensitive_file_access,
        "job_hunting_website_access": job_hunting_website_access,
        "leak_website_access": leak_website_access,
        "job_hunting_email": job_hunting_email,
        "discontent_email": discontent_email
    }

# === Step 2: KB Matching ===
def match_kb_signals(new_signals, kb, top_k=3):
    scored_matches = []
    for entry in kb:
        entry_signals = entry["signals"]
        # Count all signals that match (True-True or False-False)
        match_score = sum(new_signals.get(k) == entry_signals.get(k) for k in new_signals)
        scored_matches.append((entry, match_score))
    
    # Sort by most matching signals (highest score)
    return sorted(scored_matches, key=lambda x: -x[1])[:top_k]

def format_kb_examples(matches):
    lines = []
    for entry, _ in matches:
        user_id = entry.get("user_id", "unknown")
        risk = entry.get("risk_level", "Unknown")
        explanation = entry.get("explanation", "No explanation available.")
        signals_list = [k.replace("_", " ").title() for k, v in entry["signals"].items() if v]
        signals_str = ", ".join(signals_list) or "None"
        lines.append(f"""- User {user_id} (Risk: {risk})  \
  Signals: {signals_str}  \
  Explanation: {explanation}""")
    return "\n\n".join(lines)

# === Step 3: Load Files ===
with open("user_prompts.json", "r", encoding="utf-8") as f:
    prompts = json.load(f)

with open("knowledge_base.json", "r", encoding="utf-8") as f:
    kb = json.load(f)

# === Step 4: Inference with CoT ===
results = []

for i, prompt in enumerate(prompts):
    activity_text = prompt["content"]
    signals = extract_signals(activity_text)

    signal_summary = ", ".join([k.replace("_", " ").title() for k, v in signals.items() if v]) or "None"

    kb_matches = match_kb_signals(signals, kb)
    kb_context = format_kb_examples(kb_matches) if kb_matches else "None available"

    system_prompt = {
        "role": "system",
        "content": f"""
You are a cybersecurity analyst. Think step by step to determine if a user is an insider threat.

You will receive:
- Extracted behavioral signals
- 3 examples of prior users with risk levels, similar signals and explanations
- A full activity log

Lets think step by step, your job is to:
1. Analyze each signal one at a time and discuss its suspiciousness.
2. Compare the signal to similar users from the examples.
3. Conclude with a Risk Level (High/Low)
4. Give a short explanation of your reasoning.

Begin your response with:
User analyzed:
[User ID]
Chain of Thought:
[Your reasoning]

Then output:
Risk Level: [High or Low]  
Explanation: [brief rationale]

Signals detected in this user's activity:
{signal_summary}

Examples of similar past cases:
{kb_context}
"""
    }

    try:
        response = openai.chat.completions.create(
            model="ft:gpt-3.5-turbo-0125:personal::BWhcGDST",
            messages=[system_prompt, prompt],
            temperature=0
        )
        reply = response.choices[0].message.content

        print(f"\nPrompt {i+1} Response:\n{reply}")
        results.append({
            "input": prompt,
            "signals": signals,
            "kb_matches": [m[0] for m in kb_matches] if kb_matches else [],
            "response": reply
        })

    except Exception as e:
        print(f"Error on prompt {i+1}: {e}")

# === Step 5: Save Results ===
with open("results_with_cot_and_kb.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("\nAll responses saved to 'results_with_cot_and_kb.json'")


Prompt 1 Response:
User analyzed: AJR0319
Chain of Thought: 
Normal login and logout activity during business hours. File access and website visits seem work-related.

Signals detected in this user's activity:
None

Examples of similar past cases:
- User AMD0077 (Risk: Low)    Signals: None    Explanation: Lets think step by step. User AMD0077 is flagged as Low. No suspicious behavior was detected.

- User AMM0014 (Risk: Low)    Signals: None    Explanation: Lets think step by step. User AMM0014 is flagged as Low. No suspicious behavior was detected.

- User ATE0869 (Risk: Low)    Signals: None    Explanation: Lets think step by step. User ATE0869 is flagged as Low. No suspicious behavior was detected.

Risk Level: Low
Explanation: Normal activity detected. No insider threat detected.

Prompt 2 Response:
User analyzed: BTL0226
Chain of Thought: 
1. Usb Outside Hours - Suspicious
2. Odd Hours Logins - Suspicious
3. Leak Website Access - Suspicious

Comparing with similar examples:
- Us